In [ ]:
import twitter
import os
import importlib 
import pandas as pd
import numpy as np
from twitter.scraper import *
from IPython.display import display, HTML
import emoji
import re, pandas as pd, numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, OpenAI
from sentence_transformers import SentenceTransformer
import openai
from pprint import pprint

from tqdm.notebook import tqdm
pd.set_option("display.max_colwidth", None)  # show full text in each cell



In [78]:
importlib.reload(twitter)
from twitter import *
from datetime import datetime
def tweet_score(tweet, decay=0.7):
    # decay=1.0: linear penalty → each extra day reduces score proportionally.
    # Smaller decay (e.g. 0.5) → recency matters less.
    # Larger decay (e.g. 2) → recency matters more strongly.
    created_at = datetime.strptime(tweet["created_at"], "%a %b %d %H:%M:%S %z %Y")
    created_at = created_at.replace(tzinfo=None)  # drop tz for consistency

    # age in days (float)
    age_days = (datetime.now() - created_at).days + 1   # +1 to avoid div by 0

    # score: replies penalized by age
    return tweet["replies"] / (age_days ** decay)

rocket_lab = '91145174'
peter_beck = '976574172468936704'
nvidia= '61559439'
elon_musk = '44196397'

tweets = get_user_tweets(elon_musk, period='month=1')[0]

top_20_tweets = sorted(tweets, key=lambda x: tweet_score(x), reverse=True)[:20]
comments = get_comments(top_20_tweets[0]["tweet_id"], minimum_comments=200)[0]
original_tweet = comments[0]
comments = comments[1:]


Getting tweets for user_id:  44196397


In [ ]:
decay = 0.7
for t in sorted(tweets, key=lambda x: tweet_score(x,decay), reverse=True)[:20]:
    print(t["created_at"], t["replies"])
    print(f"{tweet_score(t, decay):.2f}")

845
Sun Sep 21 22:52:42 +0000 2025 37577
23131.36
Mon Sep 22 00:08:00 +0000 2025 33681
20733.09
Sun Sep 21 17:45:48 +0000 2025 31414
14559.23
Sun Sep 21 18:15:46 +0000 2025 26106
12099.17
Tue Sep 23 14:47:39 +0000 2025 11973
11973.00
Sun Sep 21 01:22:43 +0000 2025 23837
11047.57
Tue Sep 23 03:55:53 +0000 2025 10560
10560.00
Sun Sep 21 23:36:42 +0000 2025 16176
9957.50
Tue Sep 23 15:07:30 +0000 2025 9190
9190.00
Thu Sep 04 15:10:42 +0000 2025 64407
7910.65
Sun Sep 21 01:14:00 +0000 2025 15554
7208.70
Fri Sep 19 22:24:52 +0000 2025 22209
7198.63
Wed Sep 10 19:26:06 +0000 2025 38409
6055.41
Sat Sep 20 05:12:44 +0000 2025 15879
6017.02
Sun Sep 14 16:43:17 +0000 2025 29004
5787.06
Mon Sep 08 06:04:10 +0000 2025 36699
5269.51
Sat Sep 13 04:29:15 +0000 2025 25772
4810.31
Sun Sep 14 05:34:00 +0000 2025 23151
4619.23
Mon Sep 08 05:30:56 +0000 2025 31123
4468.87
Mon Sep 15 07:48:09 +0000 2025 20784
4464.36


In [ ]:
cleaned_tweets = []
def clean_tweet(t):
    t = re.sub(r"http\S+", "", t)  
    t = re.sub(r"\s+", " ", t).strip()
    if t.startswith('@'):
        parts = t.split(" ", 1)
        if len(parts) > 1:
            return parts[1]
        else:
            return t
    return t
    
def stringify_tweet(tweet: Tweet):
    full_tweet = []
    cleaned_tweet_text = clean_tweet(tweet["text"])
    if cleaned_tweet_text: 
        full_tweet.append(cleaned_tweet_text)
    if tweet["post_image_description"]:
        full_tweet.append(tweet["post_image_description"])
    if tweet["post_video_description"]:
        full_tweet.append(tweet["post_video_description"])
    
    if tweet["quoted_tweet"]:
        full_tweet+= [stringify_tweet(tweet["quoted_tweet"])]
    return "\n".join(full_tweet)


print(len(tweets))
for tweet in tweets:
    if tweet["retweeted_tweet"]: continue
    full_tweet = stringify_tweet(tweet)
    if not full_tweet: continue
    cleaned_tweets.append(full_tweet)
    print(full_tweet)
    print("-"*100)
print("Newest Tweet Date: ", tweets[0]["created_at"])
print("Oldest Tweet Date: ", tweets[-1]["created_at"])
print(f"{len(cleaned_tweets)}/{len(tweets)}")

845
RT @amXFreeze: Start a fresh Thread in Grok without disrupting your current chat Seamlessly continue where you left off while exploring ne…
Two windows on a computer screen displaying a chat interface. The left window shows a list of options and a chat with text about calculating a mathematical expression. The right window shows a blank thread interface with a text input area labeled "Start a Thread" and a reply section labeled "Reply to Thread." No watermarks from platforms like Instagram, TikTok, or Xiaohongshu are present.
----------------------------------------------------------------------------------------------------
RT @techdevnotes: xAI is building a Team for AI-native knowledge base/search engine ...
Black background with white text listing focus areas for xAI. Text includes bullet points about developing and optimizing search pipelines for LLM agents, designing efficient search algorithms, integrating advanced search systems, minimizing hallucinations, and ensuring robu

In [86]:
cleaned_comments = []
clean_comments_obj = []

for comment in comments:
    cleaned_comment = stringify_tweet(comment)
    if not cleaned_comment: continue
    cleaned_comments.append(cleaned_comment)
    clean_comments_obj.append(comment)
    # print(cleaned_comment)
    # print("-"*100)

print("Oldest Comment Date: ", comments[0]["created_at"])
print("Newest Comment Date: ", comments[-1]["created_at"])
print(f"{len(cleaned_comments)}/{len(comments)}")
print(len(clean_comments_obj))



Oldest Comment Date:  Mon Sep 22 02:19:48 +0000 2025
Newest Comment Date:  Sun Sep 21 23:23:55 +0000 2025
222/222
222


In [37]:
roberta_model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline("sentiment-analysis", model=roberta_model_path, tokenizer=roberta_model_path)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [84]:

roberta_sentiments= sentiment_task(cleaned_comments)


In [ ]:
roberta_df = pd.DataFrame()
roberta_df["label"] = [labels[sentiment["label"]] for sentiment in roberta_sentiments]
roberta_df["sentiment"] = [sign_map[sentiment["label"]] for sentiment in roberta_sentiments]
roberta_df["confidence"] = [sentiment["score"] for sentiment in roberta_sentiments]
roberta_df["comment"] = cleaned_comments
roberta_df["likes"] = [c["likes"] for c in clean_comments_obj]
likes = roberta_df["likes"].astype(float)
M = np.log1p(likes.max()) if likes.max() > 0 else 1.0
norm_log = np.log1p(likes) / M
baseline = 0.3
roberta_df["weight_likes"] = baseline + (1 - baseline) * norm_log


sign_map = {"positive": 1.0, "neutral": 0.0, "negative": -1.0}
labels = {"negative": "❌", "neutral": "😐", "positive": "✅"}

roberta_df["weighted"] = roberta_df["confidence"] * roberta_df["weight_likes"] + 0.5
# normalise weighted to 0-1
roberta_df["weighted"] = (roberta_df["weighted"] - roberta_df["weighted"].min()) / (roberta_df["weighted"].max() - roberta_df["weighted"].min()) * 100 * roberta_df["sentiment"]
non_neutral = roberta_df["sentiment"] != 0

# print(roberta_sentiments)
# print(aggregate_label_conf(roberta_sentiments))

print(roberta_df.groupby("label").size())
# print(non_neutral.groupby("label").size())

pos = (roberta_df["sentiment"] > 0).sum()
neg = (roberta_df["sentiment"] < 0).sum()
neu = (roberta_df["sentiment"] == 0).sum()
total = len(roberta_df)

pprint({
    "pos%": float(pos/total)*100,
    "neg%": float(neg/total)*100,
    "neu%": float(neu/total)*100,
})
print("Excluding neutral comments")
pprint({
    "pos%": float(pos/(total-neu))*100,
    "neg%": float(neg/(total-neu))*100,
})
negative_comments = roberta_df[roberta_df["sentiment"] == "❌"]
positive_comments = roberta_df[roberta_df["sentiment"] == "✅"]
neutral_comments = roberta_df[roberta_df["sentiment"] == "😐"]
roberta_df.sort_values(by="likes", ascending=False, inplace=True)

# display(HTML(negative_comments.to_html(index=False)))
# display(HTML(positive_comments.to_html(index=False)))
# display(HTML(neutral_comments.to_html(index=False)))
display(HTML(roberta_df.to_html(index=False)))
# display(HTML(roberta_df[non_neutral].to_html(index=False)))


222
label
✅    107
❌      8
😐    107
dtype: int64
{'neg%': 3.6036036036036037, 'neu%': 48.1981981981982, 'pos%': 48.1981981981982}
Excluding neutral comments
{'neg%': 6.956521739130435, 'pos%': 93.04347826086956}


label,sentiment,confidence,comment,likes,weight_likes,weighted
✅,1.0,0.785305,"Love to see it!! 🙏🙏\nDonald Trump and Elon Musk sitting and talking in a room. Both are wearing suits, with Donald Trump in a dark suit and red tie, and Elon Musk in a dark suit and tie. The setting appears to be an indoor event with other people visible in the background.",30395,1.000000,87.248865
✅,1.0,0.832993,"Glad to see it!\nElon Musk lying on a bed, holding a phone, smiling, with text overlay ""You hang up."" Donald Trump lying on a bed, holding a phone, smiling, with text overlay ""No, you hang up."" Both are depicted in casual attire, with feet visible at the edges of the beds.",20600,0.973622,91.680666
😐,0.0,0.648156,"For Charlie ❤\nFour-panel comic with a man in a suit and tie next to Jesus Christ, both with halos, against a blue sky background. In the first panel, the man asks, ""They aren\'t rioting, are they?"" In the second, Jesus replies, ""No Charlie, they\'re singing."" In the third, the man says, ""I knew it."" In the fourth, both smile. Watermarks ""RISEATURE"" appear on each panel.",18330,0.965704,0.000000
❌,-1.0,0.509637,"You’re cool with a pedophile again for Charlie?\nA screenshot of a social media post from X. The post includes text attributed to Elon Musk, mentioning Donald Trump and the Epstein files. Visible elements include Elon Musk\'s profile picture, a blue checkmark, and text with timestamps, view counts, and engagement metrics like likes and retweets.",16926,0.960301,-36.252270
✅,1.0,0.606751,"He would love this..\nDonald Trump sits in a row of seats, wearing a dark suit and red tie, positioned among other individuals in formal attire. Elon Musk, also in a dark suit, is seated nearby, engaging in conversation. The group is positioned behind a glass barrier, overlooking an indoor arena with bright lights and a red pendant lamp visible. A white railing and part of a hockey rink are seen in the foreground. The Fox News logo and text ""2024"" are displayed at the bottom of the frame.",16557,0.958806,52.168445
✅,1.0,0.779381,"Bro 🥹 I'm so happy right now\nSix panels arranged in two columns and three rows. Left column shows Elon Musk in a suit, with closed eyes in the top and middle panels, and a neutral expression in the bottom panel. Right column shows Donald Trump in a suit with an American flag background, with closed eyes in the top panel, a neutral expression in the middle panel, and a sad expression in the bottom panel. Text overlays include ""Close ur eyes bro"", ""Ok bjo"", ""Wat u see bro?"", ""Nothin bro"", ""That\'s my world without u bro"", and ""bro..."".",16298,0.957737,80.551029
✅,1.0,0.887152,"Thank you, CHARLIE\nDonald Trump and Elon Musk standing close, shaking hands. Both are wearing suits, with Donald Trump in a red tie and Elon Musk in a dark tie. A crowd is visible in the background.",13088,0.942862,96.065550
😐,0.0,0.850783,"For Charlie.\nA screenshot of a Twitter post by Charlie Kirk. The profile picture shows Charlie Kirk, a man with short dark hair. The post text is visible, stating, ""If you\'re 18 and think you\'re the opposite sex, you have a mental illness. If you\'re 8 and think you\'re the opposite sex, your mother has a mental illness."" A blue checkmark is next to the username @charliekirk11.",12800,0.941354,0.000000
✅,1.0,0.797312,🔥,9784,0.923133,78.755754
😐,0.0,0.514608,"Love to see it. 🤝\nA blurry video shows several individuals seated in an audience, including Donald Trump wearing a red tie and dark suit, flanked by others in suits. The setting appears to be a large indoor event with a crowd in the background. Text overlays on the screen read ""GLENDALE, AZ LIVE 1:29 PM MT"" and ""TURNING POINT USA"" at the top, with a Fox News graphic at the bottom stating ""THOUSANDS PAY TRIBUTE AT MEMORIAL FOR CHARLIE KIRK"" and ""Remembering Charlie Kirk."" The image is slightly distorted, with a hazy effect over the figures.",7974,0.909262,0.000000


In [ ]:
client = openai.OpenAI()
representation_model = OpenAI(client, model="gpt-4o", chat=True)
openai_BertTopic_model = BERTopic(representation_model=representation_model)

representation_model = KeyBERTInspired()
keybert_BertTopic_model = BERTopic(representation_model=representation_model)

In [ ]:
# BERTopic fitting
keybert_topics, keybert_probs = keybert_BertTopic_model.fit_transform(cleaned_tweets)
keybert_topic_info = keybert_BertTopic_model.get_topic_info()  # topic list with sizes
keybert_topic_info


openai_topics, openai_probs = openai_BertTopic_model.fit_transform(cleaned_tweets)
openai_topic_model = openai_BertTopic_model.reduce_topics(cleaned_tweets, nr_topics="auto")
openai_topic_info = openai_BertTopic_model.get_topic_info()  # topic list with sizes
openai_topic_info

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
vader_df = pd.DataFrame()
vader_df["comment"] = cleaned_comments
vader_sentiments = [analyzer.polarity_scores(sentence) for sentence in cleaned_comments]
# vader_df["neg"] = [sentiment["neg"] for sentiment in vader_sentiments]
# vader_df["neu"] = [sentiment["neu"] for sentiment in vader_sentiments]
# vader_df["pos"] = [sentiment["pos"] for sentiment in vader_sentiments]
# vader_df["compound"] = [sentiment["compound"] for sentiment in vader_sentiments]
vader_df["overall"] = [["❌", "😐", "✅"][np.argmax([sentiment["neg"], sentiment["neu"], sentiment["pos"]])] for sentiment in vader_sentiments]
vader_df["confidence"] = [sentiment[["neg", "neu", "pos"][np.argmax([sentiment["neg"], sentiment["neu"], sentiment["pos"]])]] for sentiment in vader_sentiments]
print(vader_df.groupby("overall").size())
vader_df